In [0]:
%run
"/Users/yeonmi.hwang@providence.org/Cloned/Clinical Concepts 12132020/Utilities/general"

In [0]:
# social status functions 

def get_social_status_df(df_pat, filter_string):
  df_sh_enc = spark.sql(
  """
  SELECT
      sh.pat_id,
      sh.instance,
      sh.istobaccouser as is_tobacco_user,
      sh.isalcoholuser as is_alcohol_user,
      sh.isillegaldruguser as is_illegal_drug_user,
      sh.isivdruguser as is_iv_drug_user,
      sh.years_education,
      e.contact_date
  FROM rdp_phi.socialhistory AS sh
  LEFT JOIN rdp_phi.encounter AS e
  ON sh.pat_id = e.pat_id 
    AND sh.instance = e.instance
    AND sh.pat_enc_csn_id = e.pat_enc_csn_id 
  """)
  df_social_status = df_pat.join(df_sh_enc, ['pat_id', 'instance'], how='inner').filter(filter_string).select(['pat_id','instance','lmp','is_tobacco_user','is_illegal_drug_user','is_alcohol_user','is_iv_drug_user','years_education','contact_date'])
  df_social_status.createOrReplaceTempView("social_status")
  return(df_social_status)


def smoker_status(smoking):
    yes = ['Yes']
    if (smoking in yes):
      return 1
    else:
      return 0
smoker_status_udf = F.udf(lambda smoking: smoker_status(smoking), IntegerType())
def alcohol_user_status(alcohol):
    yes = ['Yes']
    if (alcohol in yes):
      return 1
    else:
      return 0
alcohol_user_status_udf = F.udf(lambda alcohol: alcohol_user_status(alcohol), IntegerType())
def illegal_drug_user_status(il_drug):
    yes = ['Yes']
    if (il_drug in yes):
      return 1
    else:
      return 0
illegal_drug_user_status_udf = F.udf(lambda il_drug: illegal_drug_user_status(il_drug), IntegerType())


def add_drug_use_to_df(df, filter_string = "contact_date >= date_sub(lmp, 365) AND contact_date < ob_delivery_delivery_date"):
  print('Re:trieving patient social history info...')
  sh = get_social_status_df(df, filter_string).select(['pat_id', 'instance', 'lmp', 'is_tobacco_user', 'is_illegal_drug_user', 'is_alcohol_user'])
  write_data_frame_to_sandbox(sh, "yh_maternity_social_temp", sandbox_db='rdp_phi_sandbox', replace=True)
  social_temp = spark.sql("SELECT * FROM rdp_phi_sandbox.yh_maternity_social_temp")
  print('Adding smoker status...')
  social_temp = social_temp.withColumn('smoker', smoker_status_udf(F.col('is_tobacco_user')))
  print('Adding illegal drug user status...')
  social_temp = social_temp.withColumn('illegal_drug_user', illegal_drug_user_status_udf(F.col('is_illegal_drug_user')))
  print('Adding alcohol user status...')
  social_temp = social_temp.withColumn('alcohol_user', alcohol_user_status_udf(F.col('is_alcohol_user')))
  partition_columns = ['pat_id','instance','lmp']
  agg_columns = {'smoker': 'max', 
                 'illegal_drug_user': 'max', 
                'alcohol_user': 'max'}
  social_agg = aggregate_data(social_temp, partition_columns = partition_columns, aggregation_columns  =  agg_columns)
  result_df = df.join(social_agg, ['pat_id', 'instance', 'lmp'], "left")
  result_df = result_df.withColumnRenamed('smoker_max', 'smoker').withColumnRenamed('illegal_drug_user_max', 'illegal_drug_user').withColumnRenamed('alcohol_user_max', 'alcohol_user')
  return(result_df)

In [0]:
spark.sql("REFRESH TABLE rdp_phi_sandbox.yh_depression_ad_org_epicohort_final")
spark.sql("REFRESH TABLE rdp_phi_sandbox.yh_depression_ad_any_epicohort_final")
original_df = spark.sql("SELECT * FROM rdp_phi_sandbox.yh_depression_ad_org_epicohort_final")
any_df = spark.sql("SELECT * FROM rdp_phi_sandbox.yh_depression_ad_any_epicohort_final")

In [0]:
parity = spark.sql("SELECT * FROM rdp_phi_sandbox.yh_maternity_gpal")

In [0]:
original_df = original_df.join(parity, ['pat_id', 'instance', 'lmp', 'ob_delivery_delivery_date'], 'left')
#any_df = any_df.join(parity, ['pat_id', 'instance', 'lmp', 'ob_delivery_delivery_date'], 'left')
original_any_df = original_df.join(any_df.select(['pat_id', 'instance', 'lmp', 'any_depression']), how = 'left', on = ['pat_id', 'instance', 'lmp']).drop(*['illegal_drug_user', 'smoker'])
original_any_drug_df = add_drug_use_to_df(original_any_df, filter_string = "contact_date >= lmp AND contact_date < ob_delivery_delivery_date")

In [0]:
def exposure_group_sens(exp1,exp2,exp3):
  if (exp2 == 1 or exp3 == 1) and (exp1 == 1):
    return "both_exposed"
  elif (exp2 == 1 or exp3 == 1): 
    return "late_only"
  elif exp1 == 1: 
    return "early_only"
  else:
    return "non_exposed"
def exposure_group(exp1,exp2,exp3):
  if (exp2 == 1 or exp3 == 1): 
    return "late"
  elif exp1 == 1: 
    return "early_only"
  else:
    return "non_exposed"

def subgroup_exposure_group(exp1,exp2,exp3, ssri_exp1, ssri_exp2, ssri_exp3):
  if (exp2 == 1 or exp3 == 1): 
    return "late"
  elif exp1 == 1: 
    return "early_only"
  elif (ssri_exp2 == 1 or ssri_exp3 == 1):
    return "lateSSRI"
  elif (ssri_exp1 == 1):
    return "earlyonlySSRI"
  else:
    return "non_exposed"


exposure_group_sens_udf = F.udf(lambda exp1, exp2, exp3: exposure_group_sens(exp1, exp2, exp3), StringType())
exposure_group_udf = F.udf(lambda exp1, exp2, exp3: exposure_group(exp1, exp2, exp3), StringType())
subgroup_exposure_group_udf = F.udf(lambda exp1, exp2, exp3, ssri_exp1, ssri_exp2, ssri_exp3: subgroup_exposure_group(exp1,exp2,exp3, ssri_exp1, ssri_exp2, ssri_exp3), StringType())



def map_ethnicity(ethnicity):
  if (ethnicity is None):
    return None
  elif (ethnicity == 'Hispanic or Latino'):
    return 'Hispanic_Latino'
  elif (ethnicity == 'Patient Refused') | (ethnicity == 'Declined') | (ethnicity == 'Unknown'):
    return 'Unknown_NotReported'
  else:
    return 'Not_Hispanic_Latino'
    
#Define UDF function for mapping to age_group
map_ethnicity_udf = F.udf(lambda ethnicity: map_ethnicity(ethnicity), StringType())


def map_race(maternal_race):
  asian = ['Asian']
  white = ['White or Caucasian', 'White']
  african_american = ['Black or African American']
  pacific = ['Native Hawaiian or Other Pacific Islander']
  native = ['American Indian or Alaska Native']
  Other = ['Patient Refused', 'Unable to Determine', 'Unknown', 'Declined', 'Other']
  if (maternal_race is None):
    return None
  elif (len(maternal_race) > 1 ):
    return 'Multirace'
  elif (maternal_race[0] in white):
    return 'White_Caucasian'
  elif (maternal_race[0] in asian):
    return 'Asian'
  elif (maternal_race[0] in pacific):
    return 'NativeHawaiian_OtherPacificIslander'
  elif (maternal_race[0] in african_american):
    return 'Black_AfricanAmerican'
  elif (maternal_race[0] in native):
    return 'AmericanIndian_AlaskaNative'
  elif (maternal_race[0] in Other):
    return 'Unknown_NotReported'
  else:
    return None
    
#Define UDF function for mapping to age_group
map_to_race_udf = F.udf(lambda race: map_race(race), StringType())


mental_cc_list = ['mental_disorder','2yr_depression','bipolar','adjustment_disorder','anxiety_disorder','psychosis_disorder']
def other_mental_disorder(main, md1, md2, md3, md4, md5):
  if main  == 1 and (md1 == 1 or md2 == 1 or md3 == 1 or md4 == 1 or md5 == 1):
    return 0
  elif main == 1 and (md1 == 0 and md2 == 0 and md3 == 0 and md4 == 0 and md5 == 0):
    return 1 
  else: 
    return main
other_mental_disorder_udf = F.udf(lambda main, md1, md2, md3, md4, md5: other_mental_disorder(main, md1, md2, md3, md4, md5), StringType())


def categorize_BMI(bmi):
  if bmi is None:
    return None
  elif 5 < bmi < 18.5:
    return 'underweight'
  elif (bmi >= 18.5) & (bmi < 24.9):
    return 'normal'
  elif (bmi >= 25.0) & (bmi < 30):
    return 'overweight'
  elif ( 70 >bmi >= 30):
    return 'obese'
  else:
    return None
categorize_BMI_udf = F.udf(lambda bmi: categorize_BMI(bmi), StringType())

def group_insurance(insurance):
  if insurance is None:
    return None
  elif insurance == 'Medicare':
    return 'Medicaid_Medicare'
  elif insurance == 'Medicaid':
    return 'Medicaid_Medicare'
  elif insurance == 'Commercial':
    return 'Commercial'
  else:
    return None
group_insurance_udf = F.udf(lambda insurance: group_insurance(insurance), StringType())

def group_parity(parity):
  if parity is None:
    return None
  elif float(parity) >= 5:
    return 'grand_multipara'
  elif float(parity) == 0:
    return 'nullipara'
  elif 0 < float(parity) < 5:
    return 'multipara'
  else:
    return None
group_parity_udf = F.udf(lambda parity: group_parity(parity), StringType())




In [0]:
any_before = original_any_df.filter(F.col('any_depression')==1)
here = any_before.withColumn("deliveryYear", F.year(F.col('ob_delivery_delivery_date'))).filter((F.col('deliveryYear') < 2021) & (F.col('deliveryYear') > 2012))
here.filter(F.col('insurance').isNull()).count()

In [0]:
def make_epicohort_df(cohort_df):
  exposure_columns = [i for i in cohort_df.columns if 'exposure' in i] + ['any_depression']
  cohort_df = cohort_df.fillna(0, subset = exposure_columns)
  cohort_df = cohort_df.filter(((F.col('insurance').isNull())|(F.col('insurance') != "Uninsured-Self-Pay")) & (F.col('psychosis_disorder') == 0) & (F.col('bipolar') == 0))
  cohort_df = cohort_df.withColumn("deliveryYear", F.year(F.col('ob_delivery_delivery_date'))).filter((F.col('deliveryYear') < 2021) & (F.col('deliveryYear') > 2012))
  cohort_df = cohort_df.filter((F.col('ob_hx_infant_sex') == 'Female') | (F.col('ob_hx_infant_sex') == 'Male') | (F.col('ob_hx_infant_sex').isNull())) 
  cohort_df = cohort_df.withColumn('SSRI_exposure_group', exposure_group_udf(F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('SSRI_exposure_group_sens', exposure_group_sens_udf(F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('N06AA_exposure_group', exposure_group_udf(F.col("N06AA_exposure1"), F.col("N06AA_exposure2"), F.col("N06AA_exposure3"))) \
  .withColumn('N06AF_exposure_group', exposure_group_udf(F.col("N06AF_exposure1"), F.col("N06AF_exposure2"), F.col("N06AF_exposure3"))) \
  .withColumn('N06AG_exposure_group', exposure_group_udf(F.col("N06AG_exposure1"), F.col("N06AG_exposure2"), F.col("N06AG_exposure3"))) \
  .withColumn('N06AX_exposure_group', exposure_group_udf(F.col("N06AX_exposure1"), F.col("N06AX_exposure2"), F.col("N06AX_exposure3"))) \
  .withColumn('N06AA_extra_exposure_group', subgroup_exposure_group_udf(F.col("N06AA_exposure1"), F.col("N06AA_exposure2"), F.col("N06AA_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('N06AF_extra_exposure_group', subgroup_exposure_group_udf(F.col("N06AF_exposure1"), F.col("N06AF_exposure2"), F.col("N06AF_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('N06AG_extra_exposure_group', subgroup_exposure_group_udf(F.col("N06AG_exposure1"), F.col("N06AG_exposure2"), F.col("N06AG_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('N06AX_extra_exposure_group', subgroup_exposure_group_udf(F.col("N06AX_exposure1"), F.col("N06AX_exposure2"), F.col("N06AX_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('citalopram_exposure_group', subgroup_exposure_group_udf(F.col("citalopram_exposure1"), F.col("citalopram_exposure2"), F.col("citalopram_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('escitalopram_exposure_group', subgroup_exposure_group_udf(F.col("escitalopram_exposure1"), F.col("escitalopram_exposure2"), F.col("escitalopram_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('fluoxetine_exposure_group', subgroup_exposure_group_udf(F.col("fluoxetine_exposure1"), F.col("fluoxetine_exposure2"), F.col("fluoxetine_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('paroxetine_exposure_group', subgroup_exposure_group_udf(F.col("paroxetine_exposure1"), F.col("paroxetine_exposure2"), F.col("paroxetine_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('sertraline_exposure_group', subgroup_exposure_group_udf(F.col("sertraline_exposure1"), F.col("sertraline_exposure2"), F.col("sertraline_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('fluvoxamine_exposure_group', subgroup_exposure_group_udf(F.col("fluvoxamine_exposure1"), F.col("fluvoxamine_exposure2"), F.col("fluvoxamine_exposure3"), F.col("N06AB_exposure1"), F.col("N06AB_exposure2"), F.col("N06AB_exposure3"))) \
  .withColumn('ethnic_group', map_ethnicity_udf(F.col('ethnic_group'))) \
  .withColumn('Parity_group', group_parity_udf(F.col('Parity'))) \
  .withColumn('insurance_group', group_insurance_udf(F.col('insurance'))) \
  .withColumn('BMI_category', categorize_BMI_udf(F.col('pregravid_bmi'))) \
  .withColumn('race_group', map_to_race_udf(F.col('race')))
  return cohort_df

In [0]:
original_any_epi_df = make_epicohort_df(original_any_drug_df)

In [0]:
write_data_frame_to_sandbox(original_any_epi_df, "yh_depression_original_epicohort_final_final", sandbox_db='rdp_phi_sandbox', replace=True)
# write_data_frame_to_sandbox(any_epi_df, "yh_depression_any_epicohort_final_final", sandbox_db='rdp_phi_sandbox', replace=True)

In [0]:

spark.sql("REFRESH TABLE rdp_phi_sandbox.yh_depression_original_epicohort_final_final")

original_epicohort = spark.sql("SELECT * FROM rdp_phi_sandbox.yh_depression_original_epicohort_final_final").distinct()

In [0]:

original_pd = original_epicohort.toPandas()

In [0]:
def change_type_pd(cohort_pd):
  cohort_pd['deliveryYear'] = cohort_pd['deliveryYear'].astype(str)
  cohort_pd['alcohol_user'] = cohort_pd['alcohol_user'].astype(str)
  cohort_pd['prior_phq9_score'] = cohort_pd['prior_phq9_score'].astype(float)
  cohort_pd['during_phq4_score'] = cohort_pd['during_phq4_score'].astype(float)
  cohort_pd['pregravid_bmi'] = cohort_pd['pregravid_bmi'].astype(float)
  cohort_pd['Preterm_history'] = cohort_pd['Preterm_history'].astype(float)
  cohort_pd['Parity'] = cohort_pd['Parity'].astype(float)
  cohort_pd['preterm_category'] = cohort_pd['preterm_category'].map({'preterm': 1, 'term': 0})
  return cohort_pd
original_pd = change_type_pd(original_pd)

In [0]:
def simple_impute_missing_data(df):
  from sklearn.impute import SimpleImputer
  columns = df.columns
  imputer = SimpleImputer(missing_values=np.nan, strategy='median')
  df_return = pd.DataFrame(data = imputer.fit_transform(df), columns=columns)
  return(df_return)

def KNN_impute_missing_data(df): 
  from sklearn.impute import KNNImputer
  imputer = KNNImputer(n_neighbors=3, weights="uniform")
  imputer.fit_transform(df)
  df_return = pd.DataFrame(imputer.transform(df), columns = df.columns)
  return (df_return)
def comparison_group(x,df): 
  cohort1 = df[(df[x]=='non_exposed') | (df[x]=='early_only')]
  cohort2 = df.copy()
  cohort3 = df[(df[x]=='non_exposed') | (df[x]=='late')]
  cohort4 = df[(df[x]=='early_only') | (df[x]=='late')]
  cohort5 = df.copy()
  #cohort6  = df[df.latepreg_depression== 1] 
  #cohort7  = df[(df.latepreg_depression== 1)&(df[x]!='early_only')]
  #cohort8  = df[(df.latepreg_depression== 1)&(df[x]!='non_exposed')] 
  newcolumn_name = x+'_boolean'
  cohort1[newcolumn_name] = cohort1[x].map({'early_only': 1, 'non_exposed': 0})
  cohort2[newcolumn_name] = cohort2[x].map({'early_only': 0, 'late': 1, 'non_exposed': 0})
  cohort3[newcolumn_name] = cohort3[x].map({'late': 1, 'non_exposed': 0})
  cohort4[newcolumn_name] = cohort4[x].map({'late': 1, 'early_only': 0})
  cohort5[newcolumn_name] = cohort5[x].map({'early_only': 1, 'late': 1, 'non_exposed': 0})
  #cohort6[newcolumn_name] = cohort6[x].map({'late': 1, 'non_exposed': 0, 'early_only': 0})
  #cohort7[newcolumn_name] = cohort7[x].map({'late': 1, 'non_exposed': 0})
  #cohort8[newcolumn_name] = cohort8[x].map({'late': 1, 'early_only': 0})
  return cohort1, cohort2, cohort3, cohort4, cohort5

def comparison_group2(x,df): 
  cohort1 = df[(df[x]=='non_exposed') | (df[x]=='both_exposed')]
  cohort2 = df[(df[x]=='non_exposed') | (df[x]=='late_only')]
  cohort3 = df[(df[x]=='both_exposed') | (df[x]=='late_only')]
  cohort4 = df[(df[x]=='early_only') | (df[x]=='late_only')]
  cohort5 = df[(df[x]=='early_only') | (df[x]=='both_exposed')]
  newcolumn_name = x+'_boolean'
  cohort1[newcolumn_name] = cohort1[x].map({'both_exposed': 1, 'non_exposed': 0})
  cohort2[newcolumn_name] = cohort2[x].map({'late_only': 1, 'non_exposed': 0})
  cohort3[newcolumn_name] = cohort3[x].map({'late_only': 1, 'both_exposed': 0})
  cohort4[newcolumn_name] = cohort4[x].map({'late_only': 1, 'early_only': 0})
  cohort5[newcolumn_name] = cohort5[x].map({'both_exposed': 1, 'early_only': 0})
  return cohort1, cohort2, cohort3, cohort4, cohort5

def subgroup_comparison_group(x, df): 
  cohort1 = df[(df[x]=='non_exposed')|(df[x]=='early_only')]
  cohort2 = df[(df[x]!='earlyonlySSRI') & (df[x]!='lateSSRI')]
  cohort3 = df[(df[x]=='non_exposed')| (df[x]=='late')]
  cohort4 = df[(df[x]=='early_only')|(df[x]=='late')]
  cohort5 = df[(df[x]!='earlyonlySSRI') & (df[x]!='lateSSRI')]
  newcolumn_name = x+'_boolean'
  cohort1[newcolumn_name] = cohort1[x].map({'early_only': 1, 'non_exposed': 0})
  cohort2[newcolumn_name] = cohort2[x].map({'early_only': 0, 'late': 1, 'non_exposed': 0})
  cohort3[newcolumn_name] = cohort3[x].map({'late': 1, 'non_exposed': 0})
  cohort4[newcolumn_name] = cohort4[x].map({'late': 1, 'early_only': 0})
  cohort5[newcolumn_name] = cohort5[x].map({'early_only': 1, 'late': 1, 'non_exposed': 0})
  return cohort1, cohort2, cohort3, cohort4, cohort5

def get_propensity_score(T_col, X_cols, df):
  from sklearn.linear_model import LogisticRegression
  ps_model = LogisticRegression(C=1e6, max_iter=10000, solver='lbfgs').fit(df[X_cols], df[T_col])
  data_ps = df.assign(propensity_score=ps_model.predict_proba(df[X_cols])[:, 1])
  return (data_ps)

def get_ps_for_all(x, cohort_dfs, data_columns):
  ps_dfs = []
  for i in cohort_dfs: 
    ps_dfs.append(get_propensity_score(x, data_columns, i))
  return ps_dfs

def unadjusted(df, x, outcome):
  import statsmodels.formula.api as smf
  model= smf.logit(formula="{0} ~ {1}".format(outcome, x), data= df).fit()
  model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
  model_odds
  return model_odds


def propensity_score_adjustment(df, x , outcome):
  import statsmodels.formula.api as smf
  model= smf.logit(formula="{0} ~ {1} + propensity_score".format(outcome, x), data= df).fit()
  model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
  model_odds
  return model_odds 

def propensity_score_matching(df, x, outcome):
  treatment = df[x] 
  mask = treatment == 1
  pscore = df['propensity_score']
  pos_pscore = np.asarray(pscore[mask])
  neg_pscore = np.asarray(pscore[~mask])
  print('treatment count:', pos_pscore.shape)
  print('control count:', neg_pscore.shape)
  from sklearn.neighbors import NearestNeighbors
  if len(neg_pscore) > len(pos_pscore):
    knn = NearestNeighbors(n_neighbors=3, metric='euclidean', n_jobs=1)
    knn.fit(neg_pscore.reshape(-1, 1))
    distances, indices = knn.kneighbors(pos_pscore.reshape(-1, 1))
    df_pos = df[mask]
    df_neg = df[~mask].iloc[indices[:, 0]]
  else: 
    knn = NearestNeighbors(n_neighbors=3, metric='euclidean', n_jobs=1)
    knn.fit(pos_pscore.reshape(-1, 1))
    distances, indices = knn.kneighbors(neg_pscore.reshape(-1, 1))
    df_pos = df[mask].iloc[indices[:, 0]] 
    df_neg = df[~mask]
  df_matched = pd.concat([df_pos, df_neg], axis=0)
  print (df_matched.shape)
  import statsmodels.formula.api as smf
  model= smf.logit(formula="{0} ~ {1}".format(outcome, x), data= df_matched).fit()
  model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
  model_odds
  return model_odds
def unadjusted_con(df, x, outcome):
  import statsmodels.formula.api as smf
  model= smf.ols(formula="{0} ~ {1}".format(outcome, x), data= df).fit()
  model_odds = pd.DataFrame(model.params, columns= ['coef'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] =model.conf_int()
  model_odds
  return model_odds


def propensity_score_adjustment_con(df, x, outcome):
  import statsmodels.formula.api as smf
  model= smf.ols(formula="{0} ~ {1} + propensity_score".format(outcome, x), data= df).fit()
  model_odds = pd.DataFrame(model.params, columns= ['coef'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] = model.conf_int()
  model_odds
  return model_odds 

def propensity_score_matching_con(df,x, outcome):
  treatment = df[outcome] 
  mask = treatment == 1
  pscore = df['propensity_score']
  pos_pscore = np.asarray(pscore[mask])
  neg_pscore = np.asarray(pscore[~mask])
  print('treatment count:', pos_pscore.shape)
  print('control count:', neg_pscore.shape)
  from sklearn.neighbors import NearestNeighbors
  if len(neg_pscore) > len(pos_pscore):
    knn = NearestNeighbors(n_neighbors=3, metric='euclidean', n_jobs=1)
    knn.fit(neg_pscore.reshape(-1, 1))
    distances, indices = knn.kneighbors(pos_pscore.reshape(-1, 1))
    df_pos = df[mask]
    df_neg = df[~mask].iloc[indices[:, 0]]
  else: 
    knn = NearestNeighbors(n_neighbors=3, metric='euclidean', n_jobs=1)
    knn.fit(pos_pscore.reshape(-1, 1))
    distances, indices = knn.kneighbors(neg_pscore.reshape(-1, 1))
    df_pos = df[mask].iloc[indices[:, 0]] 
    df_neg = df[~mask]
  df_matched = pd.concat([df_pos, df_neg], axis=0)
  print (df_matched.shape)
  import statsmodels.formula.api as smf
  model= smf.ols(formula="{0} ~ {1}".format(outcome, x), data= df_matched).fit()
  model_odds = pd.DataFrame(model.params, columns= ['coef'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] = model.conf_int()
  model_odds
  return model_odds

In [0]:
import numpy as np
def sig_check(pval):
  if pval < 1e-5:
    return ('*****')
  elif pval < 1e-4:
    return ('****')
  elif pval < 1e-3:
    return ('***')
  elif pval < 1e-2:
    return ('**')
  elif pval < 0.05:
    return ('*')
  elif pval < 0.1:
    return ('+')
  else:
    return (' ')
def unadjusted_OR(list_df, outcome, x):
  result = {}
  ref = []
  comparison = []
  ORs = []
  ps = []
  lbs = []
  ubs = []
  statistical_analysis = []
  labels = []
  significance = []
  for i in range(len(list_df)):
      model_odds = unadjusted(list_df[i], x, outcome)
      OR = model_odds.iloc[1,0]
      p = model_odds.iloc[1,1]
      lb = model_odds.iloc[1,2]
      ub = model_odds.iloc[1,3]
      label_string = str(np.round(OR, 1)) + ' [' + str(np.round(lb, 1)) + ',' + str(np.round(ub, 1)) + ']'
      ref.append(np.sum(list_df[i][x] == 0))
      comparison.append(np.sum(list_df[i][x] == 1))
      ORs.append(OR)
      ps.append(p)
      lbs.append(lb)
      ubs.append(ub)
      statistical_analysis.append('unadjusted')
      labels.append(label_string)
      significance.append(sig_check(p))
  result['reference group'] = ref
  result['comparison group'] = comparison
  result['OR'] = ORs
  result['p-value'] = ps
  result['2.5%'] = lbs
  result['97.5%'] = ubs
  result['statistical_analysis'] = statistical_analysis
  result['label'] = labels
  result['significance'] = significance
  result_df = pd.DataFrame(result)
  return result_df
def ps_adjusted_OR(list_df, outcome, x):
  result = {}
  ref = []
  comparison = []
  ORs = []
  ps = []
  lbs = []
  ubs = []
  statistical_analysis = []
  labels = []
  significance = []
  for i in range(len(list_df)):
      model_odds = propensity_score_adjustment(list_df[i], x, outcome)
      OR = model_odds.iloc[1,0]
      p = model_odds.iloc[1,1]
      lb = model_odds.iloc[1,2]
      ub = model_odds.iloc[1,3]
      label_string = str(np.round(OR, 1)) + ' [' + str(np.round(lb, 1)) + ',' + str(np.round(ub, 1)) + ']'
      ref.append(np.sum(list_df[i][x] == 0))
      comparison.append(np.sum(list_df[i][x] == 1))
      ORs.append(OR)
      ps.append(p)
      lbs.append(lb)
      ubs.append(ub)
      statistical_analysis.append('ps_adjusted')
      labels.append(label_string)
      significance.append(sig_check(p))
  result['reference group'] = ref
  result['comparison group'] = comparison
  result['OR'] = ORs
  result['p-value'] = ps
  result['2.5%'] = lbs
  result['97.5%'] = ubs
  result['statistical_analysis'] = statistical_analysis
  result['label'] = labels
  result['significance'] = significance
  result_df = pd.DataFrame(result)
  return result_df
def unadjusted_coef(list_df, outcome, x):
  result = {}
  ref = []
  comparison = []
  coefs = []
  ps = []
  lbs = []
  ubs = []
  statistical_analysis = []
  labels = []
  significance = []
  for i in range(len(list_df)):
      model_odds = unadjusted_con(list_df[i], x, outcome)
      coef = model_odds.iloc[1,0]
      p = model_odds.iloc[1,1]
      lb = model_odds.iloc[1,2]
      ub = model_odds.iloc[1,3]
      label_string = str(np.round(coef, 1)) + ' [' + str(np.round(lb, 1)) + ',' + str(np.round(ub, 1)) + ']'
      ref.append(np.sum(list_df[i][x] == 0))
      comparison.append(np.sum(list_df[i][x] == 1))
      coefs.append(coef)
      ps.append(p)
      lbs.append(lb)
      ubs.append(ub)
      statistical_analysis.append('unadjusted')
      labels.append(label_string)
      significance.append(sig_check(p))
  result['reference group'] = ref
  result['comparison group'] = comparison
  result['coef'] = coefs
  result['p-value'] = ps
  result['2.5%'] = lbs
  result['97.5%'] = ubs
  result['statistical_analysis'] = statistical_analysis
  result['label'] = labels
  result['significance'] = significance
  result_df = pd.DataFrame(result)
  return result_df
def ps_adjusted_coef(list_df, outcome, x):
  result = {}
  ref = []
  comparison = []
  coefs = []
  ps = []
  lbs = []
  ubs = []
  statistical_analysis = []
  labels = []
  significance = []
  for i in range(len(list_df)):
      model_odds = propensity_score_adjustment_con(list_df[i], x, outcome)
      coef = model_odds.iloc[1,0]
      p = model_odds.iloc[1,1]
      lb = model_odds.iloc[1,2]
      ub = model_odds.iloc[1,3]
      label_string = str(np.round(coef, 1)) + ' [' + str(np.round(lb, 1)) + ',' + str(np.round(ub, 1)) + ']'
      ref.append(np.sum(list_df[i][x] == 0))
      comparison.append(np.sum(list_df[i][x] == 1))
      coefs.append(coef)
      ps.append(p)
      lbs.append(lb)
      ubs.append(ub)
      statistical_analysis.append('ps_adjusted')
      labels.append(label_string)
      significance.append(sig_check(p))
  result['reference group'] = ref
  result['comparison group'] = comparison
  result['coef'] = coefs
  result['p-value'] = ps
  result['2.5%'] = lbs
  result['97.5%'] = ubs
  result['statistical_analysis'] = statistical_analysis
  result['label'] = labels
  result['significance'] = significance
  result_df = pd.DataFrame(result)
  return result_df

# Data analysis plan 
1. main analysis 
2. sensitivity analysis 
3. subgroup analysis on SSRI subgroups 

## sensitivity analyses
a) women with depression diagnosis anytime before the pregnancy 

b) women who have depression diagnosis during the late pregnancy

c) women who have valid PHQ-9 score during the two year prepregnancy period + adjust for the depression screening score

d) women who have valid PHQ-4 score during the late pregnancy period + adjust for the depression screening score 

e) analysis on the non-SSRI

f) drug-specific analysis 

g) late-only / early-only analysis 

### main analysis
1. unadjusted OR, gest
2. propensity score adjusted adjusted OR, gest

In [0]:
categorical_columns = ['ob_hx_infant_sex', 'ethnic_group','race_group','insurance_group','deliveryYear','Parity_group', 'BMI_category']

continuous_columns = ['age_at_start_dt', 'Preterm_history']

valid_prior_continuous_columns = ['age_at_start_dt', 'Preterm_history', 'prior_phq9_score']

valid_during_continuous_columns = ['age_at_start_dt', 'Preterm_history', 'during_phq4_score']

binary_columns = ['anxiety_disorder', 'adjustment_disorder','smoker', 'illegal_drug_user', 'alcohol_user', 'N06AF_exposure0','N06AB_exposure0','N06AA_exposure0','N06AX_exposure0', 'renal_diseases','chronic_lung_disease','diabetes','leukemia','pneumonia','sepsis','cardiovascular_diseases','anemia','sickle_cell_diseases','cystic_fibrosis', 'asthma']

columns_including  = ['pat_id', 'instance', 'gestational_days', 'preterm_category', 'latepreg_depression', 'during_phq4_score','SSRI_exposure_group', 'SSRI_exposure_group_sens','N06AA_exposure_group', 'N06AF_exposure_group', 'N06AG_exposure_group','N06AX_exposure_group','N06AA_extra_exposure_group', 'N06AF_exposure_group', 'N06AG_extra_exposure_group','N06AX_extra_exposure_group', 'citalopram_exposure_group','escitalopram_exposure_group', 'fluoxetine_exposure_group', 'paroxetine_exposure_group', 'sertraline_exposure_group', 'fluvoxamine_exposure_group', '2yr_depression', 'any_depression']

In [0]:
import numpy as np
def cohort_interest(cohort_pd,
                    categorical_columns = categorical_columns,
                    continuous_columns = continuous_columns,
                    binary_columns = binary_columns,
                    columns_including = columns_including):
  cohort_interest = cohort_pd[list(set(columns_including + categorical_columns + continuous_columns + binary_columns))]
  final_df = pd.get_dummies(cohort_interest, columns= categorical_columns, prefix=["fetal_sex", "ethnic_group", "race_group", "insurance_group", "deliveryYear", "parity_group", "BMI_group"])
  X = final_df[list(set(final_df.columns) - set(columns_including) - set(categorical_columns))]
  X_simple_imputed = simple_impute_missing_data(X).reset_index()
  imputed_df = pd.concat([final_df[columns_including].reset_index(), X_simple_imputed], axis=1)
  covariates = X.columns 
  return imputed_df, covariates 

In [0]:
a, b, c, d, e = comparison_group('SSRI_exposure_group',original_pd)

cohorts = [a, b, c, d, e]

ps_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', cohorts, covariates)


In [0]:
def unadjusted_adjusted_result(exposure_group_column, ps_cohort_list, continuous_outcome = 'gestational_days', categorical_outcome = 'preterm_category'):
  coef = unadjusted_coef(ps_cohort_list, continuous_outcome, exposure_group_column)
  adjusted_coef = ps_adjusted_coef(ps_cohort_list, continuous_outcome, exposure_group_column)
  coef_concat = pd.concat([coef, adjusted_coef], axis=0)
  OR = unadjusted_OR(ps_cohort_list, categorical_outcome, exposure_group_column)
  adjusted_OR = ps_adjusted_OR(ps_cohort_list, categorical_outcome, exposure_group_column)
  OR_concat = pd.concat([OR, adjusted_OR], axis=0)
  return OR_concat, coef_concat
def generate_result(OR, coef):
  d = {'reference group': OR['reference group'].iloc[0:5,],
       'comparison group' : OR['comparison group'].iloc[0:5,],
       'crude OR': OR['label'].iloc[0:5,]+OR['significance'].iloc[0:5,],
       'crude OR p-value': OR['p-value'].iloc[0:5,],
       'adjusted OR': OR['label'].iloc[5:10,]+OR['significance'].iloc[5:10,],
       'adjusted OR p-value': OR['p-value'].iloc[5:10,],
       'crude coef': coef['label'].iloc[0:5,]+coef['significance'].iloc[0:5,],
       'crude coef p-value': coef['p-value'].iloc[0:5,],
       'adjusted coef': coef['label'].iloc[5:10,]+coef['significance'].iloc[5:10,],
       'adjusted coef p-value': coef['p-value'].iloc[5:10,]}
  return pd.DataFrame(d)

#### sensitivity analysis a, b 
- women with depression diagnosis before the pregnancy 
- women with depression diagnosis during the pregnancy

In [0]:
anytime_before = original_pd[original_pd.any_depression == 1]
during = original_pd[original_pd.latepreg_depression == 1]
any_during = anytime_before[anytime_before.latepreg_depression == 1]

In [0]:
def propensity_score_adjustment(df, x , outcome):
  import statsmodels.formula.api as smf
  model= smf.logit(formula="{0} ~ {1} + propensity_score".format(outcome, x), data= df).fit()
  model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
  model_odds['z-value']= model.pvalues
  model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
  model_odds
  return model_odds 

In [0]:
check = ps_any_cohorts[0].reset_index()

In [0]:
any_covariates

In [0]:
import statsmodels.formula.api as smf
model= smf.logit(formula="{0} ~ {1} + propensity_score".format('preterm_category', 'SSRI_exposure_group'), data = check).fit()
model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])

In [0]:
model.summary()

In [0]:
propensity_score_adjustment(any_a, any_covariates, 'preterm_category')

In [0]:
any_cohort_df, any_covariates = cohort_interest(anytime_before)
during_cohort_df, during_covariates = cohort_interest(during)
any_during_cohort_df, any_during_covariates = cohort_interest(any_during)

any_a, any_b, any_c, any_d, any_e = comparison_group('SSRI_exposure_group',any_cohort_df)
during_a, during_b, during_c, during_d, during_e = comparison_group('SSRI_exposure_group', during_cohort_df)
any_during_a, any_during_b, any_during_c, any_during_d, any_during_e = comparison_group('SSRI_exposure_group', any_during_cohort_df)

sens_a, sens_b, sens_c, sens_d, sens_e = comparison_group2('SSRI_exposure_group_sens',any_cohort_df)



In [0]:
any_cohorts = [any_a, any_b, any_c, any_d, any_e]
during_cohorts = [during_a, during_b, during_c, during_d, during_e]
any_during_cohorts = [any_during_a, any_during_b, any_during_c, any_during_d, any_during_e]
sens_cohorts = [sens_a, sens_b, sens_c, sens_d, sens_e]
ps_sens_cohorts = get_ps_for_all('SSRI_exposure_group_sens_boolean', sens_cohorts, any_covariates)
ps_any_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', any_cohorts, any_covariates)
ps_during_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', during_cohorts, during_covariates)
ps_any_during_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', any_during_cohorts, any_during_covariates)

In [0]:
ps_adjusted_OR(ps_any_cohorts, 'SSRI_exposure_group_boolean' ,  any_covariates)

In [0]:
any_OR, any_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_any_cohorts)
during_OR, during_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_during_cohorts)
any_during_OR, any_during_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_any_during_cohorts)
sens_OR, sens_coef =  unadjusted_adjusted_result('SSRI_exposure_group_sens_boolean', ps_sens_cohorts)

In [0]:
generate_result(sens_OR, sens_coef)

In [0]:
generate_result(any_during_OR, any_during_coef)

### sensitivity analysis b 
- women who have valid phq-9 scores 
- additionally adjust for phq-9 score

In [0]:
prior_valid_df = original_pd[original_pd.prior_phq9_score.notnull()]
any_prior_valid_df = anytime_before[anytime_before.prior_phq9_score.notnull()]

prior_valid_cohort_df, prior_valid_covariates = cohort_interest(prior_valid_df, continuous_columns = valid_prior_continuous_columns)
any_prior_valid_cohort_df, any_prior_valid_covariates = cohort_interest(any_prior_valid_df, continuous_columns = valid_prior_continuous_columns)


prior_valid_a, prior_valid_b, prior_valid_c, prior_valid_d, prior_valid_e = comparison_group('SSRI_exposure_group', prior_valid_cohort_df)
any_prior_valid_a, any_prior_valid_b, any_prior_valid_c, any_prior_valid_d, any_prior_valid_e = comparison_group('SSRI_exposure_group', any_prior_valid_cohort_df)


prior_valid_cohorts = [prior_valid_a, prior_valid_b, prior_valid_c, prior_valid_d, prior_valid_e]
any_prior_valid_cohorts = [any_prior_valid_a, any_prior_valid_b, any_prior_valid_c, any_prior_valid_d, any_prior_valid_e]


ps_prior_valid_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', prior_valid_cohorts, prior_valid_covariates)
ps_any_prior_valid_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', any_prior_valid_cohorts , any_prior_valid_covariates)

In [0]:
prior_valid_OR, prior_valid_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_prior_valid_cohorts)
any_prior_valid_OR, any_prior_valid_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_any_prior_valid_cohorts)

In [0]:
generate_result(any_prior_valid_OR, any_prior_valid_coef)

### sensitivity analysis c 
- women who have PHQ-4 score during the late pregnancy 
- adjust for the PHQ-4 score during the late pregnancy

In [0]:
during_valid_df = original_pd[original_pd.during_phq4_score.notnull()]
any_during_valid_df = anytime_before[anytime_before.during_phq4_score.notnull()]

during_valid_cohort_df, during_valid_covariates = cohort_interest(during_valid_df, continuous_columns = valid_during_continuous_columns)
any_during_valid_cohort_df, any_during_valid_covariates = cohort_interest(any_during_valid_df, continuous_columns = valid_during_continuous_columns)


during_valid_a, during_valid_b, during_valid_c, during_valid_d, during_valid_e = comparison_group('SSRI_exposure_group', during_valid_cohort_df)
any_during_valid_a, any_during_valid_b, any_during_valid_c, any_during_valid_d, any_during_valid_e = comparison_group('SSRI_exposure_group', any_during_valid_cohort_df)


during_valid_cohorts = [during_valid_a, during_valid_b, during_valid_c, during_valid_d, during_valid_e]
any_during_valid_cohorts = [any_during_valid_a, any_during_valid_b, any_during_valid_c, any_during_valid_d, any_during_valid_e]


ps_during_valid_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', during_valid_cohorts, during_valid_covariates)
ps_any_during_valid_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', any_during_valid_cohorts , any_during_valid_covariates)

In [0]:
during_valid_OR, during_valid_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_during_valid_cohorts)
any_during_valid_OR, any_during_valid_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_any_during_valid_cohorts)

In [0]:
generate_result(any_during_valid_OR, any_during_valid_coef)

### sensitivity analysis d 
- N06AA exposure group 
- N06AF exposure group 
- N06AG exposure group 
- N06AX exposure group

In [0]:
any_cohort_df, any_covariates = cohort_interest(anytime_before)
any_AA_a, any_AA_b, any_AA_c, any_AA_d, any_AA_e = comparison_group('N06AA_exposure_group',any_cohort_df)
any_AX_a, any_AX_b, any_AX_c, any_AX_d, any_AX_e = comparison_group('N06AX_exposure_group',any_cohort_df)
any_noB_AA_a, any_noB_AA_b, any_noB_AA_c, any_noB_AA_d, any_noB_AA_e = subgroup_comparison_group('N06AA_extra_exposure_group',any_cohort_df)
any_noB_AX_a, any_noB_AX_b, any_noB_AX_c, any_noB_AX_d, any_noB_AX_e = subgroup_comparison_group('N06AX_extra_exposure_group',any_cohort_df)
any_AA_cohorts = [any_AA_a, any_AA_b, any_AA_c, any_AA_d, any_AA_e]
any_AX_cohorts = [any_AX_a, any_AX_b, any_AX_c, any_AX_d, any_AX_e]
any_noB_AA_cohorts = [any_noB_AA_a, any_noB_AA_b, any_noB_AA_c, any_noB_AA_d, any_noB_AA_e]
any_noB_AX_cohorts = [any_noB_AX_a, any_noB_AX_b, any_noB_AX_c, any_noB_AX_d, any_noB_AX_e]
ps_N06AA_exposure_cohorts = get_ps_for_all('N06AA_exposure_group_boolean', any_AA_cohorts, any_covariates)
ps_N06AX_exposure_cohorts = get_ps_for_all('N06AX_exposure_group_boolean', any_AX_cohorts, any_covariates)
ps_noB_N06AA_exposure_cohorts = get_ps_for_all('N06AA_extra_exposure_group_boolean', any_noB_AA_cohorts , any_covariates)
ps_noB_N06AX_exposure_cohorts = get_ps_for_all('N06AX_extra_exposure_group_boolean', any_noB_AX_cohorts , any_covariates)
any_N06AA_OR, any_N06AA_coef = unadjusted_adjusted_result('N06AA_exposure_group_boolean', ps_N06AA_exposure_cohorts)
any_N06AX_OR, any_N06AX_coef = unadjusted_adjusted_result('N06AX_exposure_group_boolean', ps_N06AX_exposure_cohorts)
any_noB_N06AA_OR, any_noB_N06AA_coef = unadjusted_adjusted_result('N06AA_extra_exposure_group_boolean', ps_noB_N06AA_exposure_cohorts)
any_noB_N06AX_OR, any_noB_N06AX_coef = unadjusted_adjusted_result('N06AX_extra_exposure_group_boolean', ps_noB_N06AX_exposure_cohorts)

In [0]:
generate_result(any_noB_N06AX_OR, any_noB_N06AX_coef)

In [0]:
generate_result(any_noB_N06AA_OR, any_noB_N06AA_coef)

### sensitivity analysis e
- exclude who were exposed to other antidepressant

In [0]:
anytime_before_SSRI_only = anytime_before[(anytime_before['N06AA_exposure_group'] == 'non_exposed') & (anytime_before['N06AX_exposure_group'] == 'non_exposed')]
any_SSRI_cohort_df, any_SSRI_covariates = cohort_interest(anytime_before_SSRI_only)
any_SSRI_a, any_SSRI_b, any_SSRI_c, any_SSRI_d, any_SSRI_e = comparison_group('SSRI_exposure_group', any_SSRI_cohort_df)
SSRI_only_cohorts = [any_SSRI_a, any_SSRI_b, any_SSRI_c, any_SSRI_d, any_SSRI_e]
ps_SSRI_only_exposure_cohorts = get_ps_for_all('SSRI_exposure_group_boolean', SSRI_only_cohorts, any_SSRI_covariates)
any_SSRI_OR, any_SSRI_coef = unadjusted_adjusted_result('SSRI_exposure_group_boolean', ps_SSRI_only_exposure_cohorts)

In [0]:
generate_result(any_SSRI_OR, any_SSRI_coef)

### SSRI subgroup analysis
- citalopram
- escitalopram 
- fluoxetine
- paroxetine
- sertraline 
- fluvoxamine

#### citalopram exposure group

In [0]:

any_cohort_df, any_covariates = cohort_interest(anytime_before)
any_citalopram_a, any_citalopram_b, any_citalopram_c, any_citalopram_d, any_citalopram_e = subgroup_comparison_group('citalopram_exposure_group',any_cohort_df)
any_escitalopram_a, any_escitalopram_b, any_escitalopram_c, any_escitalopram_d, any_escitalopram_e = subgroup_comparison_group('escitalopram_exposure_group',any_cohort_df)
any_fluoxetine_a, any_fluoxetine_b, any_fluoxetine_c, any_fluoxetine_d, any_fluoxetine_e = subgroup_comparison_group('fluoxetine_exposure_group',any_cohort_df)
any_paroxetine_a, any_paroxetine_b, any_paroxetine_c, any_paroxetine_d, any_paroxetine_e = subgroup_comparison_group('paroxetine_exposure_group',any_cohort_df)
any_sertraline_a, any_sertraline_b, any_sertraline_c, any_sertraline_d, any_sertraline_e = subgroup_comparison_group('sertraline_exposure_group',any_cohort_df)

any_citalopram_cohorts = [any_citalopram_a, any_citalopram_b, any_citalopram_c, any_citalopram_d, any_citalopram_e]
any_escitalopram_cohorts = [any_escitalopram_a, any_escitalopram_b, any_escitalopram_c, any_escitalopram_d, any_escitalopram_e]
any_fluoxetine_cohorts = [any_fluoxetine_a, any_fluoxetine_b, any_fluoxetine_c, any_fluoxetine_d, any_fluoxetine_e]
any_paroxetine_cohorts = [any_paroxetine_a, any_paroxetine_b, any_paroxetine_c, any_paroxetine_d, any_paroxetine_e]
any_sertraline_cohorts = [any_sertraline_a, any_sertraline_b, any_sertraline_c, any_sertraline_d, any_sertraline_e]

ps_citalopram_cohorts = get_ps_for_all('citalopram_exposure_group_boolean', any_citalopram_cohorts , any_covariates)
ps_escitalopram_cohorts= get_ps_for_all('escitalopram_exposure_group_boolean', any_escitalopram_cohorts, any_covariates)
ps_fluoxetine_cohorts = get_ps_for_all('fluoxetine_exposure_group_boolean', any_fluoxetine_cohorts, any_covariates)
ps_paroxetine_cohorts = get_ps_for_all('paroxetine_exposure_group_boolean', any_paroxetine_cohorts, any_covariates)
ps_sertraline_cohorts = get_ps_for_all('sertraline_exposure_group_boolean', any_sertraline_cohorts , any_covariates)


any_citalopram_OR, any_citalopram_coef = unadjusted_adjusted_result('citalopram_exposure_group_boolean', ps_citalopram_cohorts)
any_escitalopram_OR, any_escitalopram_coef = unadjusted_adjusted_result('escitalopram_exposure_group_boolean', ps_escitalopram_cohorts)
any_fluoxetine_OR, any_fluoxetine_coef = unadjusted_adjusted_result('fluoxetine_exposure_group_boolean', ps_fluoxetine_cohorts)
any_paroxetine_OR, any_paroxetine_coef = unadjusted_adjusted_result('paroxetine_exposure_group_boolean', ps_paroxetine_cohorts)
any_sertraline_OR, any_sertraline_coef = unadjusted_adjusted_result('sertraline_exposure_group_boolean', ps_sertraline_cohorts)


In [0]:
generate_result(any_paroxetine_OR, any_paroxetine_coef)

# Descriptive Statistics

In [0]:
def categorical_variable(cohort_pd, variable):
  newcount = {}
  counts = pd.crosstab(cohort_pd[variable], cohort_pd['SSRI_exposure_group'])
  variable_col = ['total N']+counts.reset_index().sort_values('index').iloc[:,0].tolist()
  non_exposed = counts.iloc[:,2]
  late = counts.iloc[:,1]
  early_only = counts.iloc[:,0]
  n_p = np.round(non_exposed/sum(non_exposed)*100,2).tolist()
  n_e = np.round(early_only/sum(early_only)*100,2).tolist()
  n_l = np.round(late/sum(late)*100,2).tolist()
  non_exposed_col = [sum(non_exposed)]
  early_only_col = [sum(early_only)]
  late_col = [sum(late)]
  for i in range(len(counts)):
    non_exposed_col.append((str(non_exposed[i])+' ('+str(n_p[i])+')'))
    early_only_col.append((str(early_only[i])+' ('+str(n_e[i])+')'))
    late_col.append((str(late[i])+' ('+str(n_l[i])+')'))
  newcount['Variable'] = variable_col
  newcount['Total'] = total_col
  newcount['Non_exposed'] = non_exposed_col
  newcount['Early_only'] = early_only_col
  newcount['Late'] = late_col
  result_df = pd.DataFrame(newcount)
  return result_df
def cohort_categorical_variable(cohort_pd, variable):
  newcount = {}
  counts = pd.DataFrame(cohort_pd[variable].value_counts())
  variable = ['total N']+counts.reset_index().iloc[:,0].tolist()
  cohort = counts.iloc[:,0]
  n_cohort = np.round(cohort/sum(cohort)*100,2).tolist()
  cohort_col = [sum(cohort)]
  for i in range(len(counts)):
    cohort_col.append((str(cohort[i])+' ('+str(n_cohort[i])+')'))
  newcount['Variable'] = variable
  newcount['cohort'] = cohort_col
  result_df = pd.DataFrame(newcount)
  return result_df
def continuous_variable(cohort_pd, variable):
  newcount = {}
  counts = cohort_pd.groupby('SSRI_exposure_group').agg({variable:['count','mean','std']}).transpose()
  variable_col = ['total N', variable]
  non_exposed = counts.iloc[:,2].tolist()
  late = counts.iloc[:,1].tolist()
  early_only = counts.iloc[:,0].tolist()
  total_col = [round(cohort_pd[variable].count(),0), str(round(cohort_pd[variable].mean(), 1))+'±'+str(round(cohort_pd[variable].std(),1))]
  non_exposed_col = [round(non_exposed[0],0),(str(round(non_exposed[1],1))+'±'+str(round(non_exposed[2],1)))]
  early_only_col = [round(early_only[0],0),(str(round(early_only[1],1))+'±'+str(round(early_only[2],1)))]
  late_col = [round(late[0],0),(str(round(late[1],1))+'±'+str(round(late[2],1)))]
  #p_col = ['', run_ANOVA_test(variable, cohort_pd)]
  newcount['Variable'] = variable_col
  newcount['Total'] = total_col
  newcount['Non_exposed'] = non_exposed_col
  newcount['Early_only'] = early_only_col
  newcount['Late'] = late_col
  #newcount['p-value'] = p_col
  result_df = pd.DataFrame(newcount)
  return result_df

def categorize_PHQ9(phq9):
  if phq9 is None:
    return None
  elif phq9 < 5:
    return 'minimal (1-4)'
  elif phq9 < 10:
    return 'mild (5-9)'
  elif phq9 < 15:
    return 'moderate (10-14)'
  elif phq9 < 20 :
    return 'moderately severe (15-19)'
  elif phq9 < 28 :
    return 'severe (20-27)'
  else:
    return None
anytime_before['phq-9 category'] = anytime_before['prior_phq9_score'].apply(lambda x: categorize_PHQ9(x))
def categorize_PHQ4(phq4):
  if phq4 is None:
    return None
  elif phq4 < 3:
    return 'normal (0-2)'
  elif phq4 < 6:
    return 'mild (3-5)'
  elif phq4 < 9:
    return 'moderate (6-8)'
  elif phq4 < 23:
    return 'severe (9-12)'
  else:
    return None
anytime_before['phq-4 category'] = anytime_before['during_phq4_score'].apply(lambda x: categorize_PHQ4(x))


In [0]:
def categorical_variable(cohort_pd, variable):
  newcount = {}
  counts = pd.crosstab(cohort_pd[variable], cohort_pd['SSRI_exposure_group'])
  variable_col = ['total N']+counts.reset_index()[variable].tolist()
  non_exposed = counts.iloc[:,2].to_numpy()
  late = counts.iloc[:,1].to_numpy()
  early_only = counts.iloc[:,0].to_numpy()
  total = non_exposed + late + early_only
  n_t = np.round(total/total.sum()*100,1).tolist()
  n_p = np.round(non_exposed/non_exposed.sum()*100,1).tolist()
  n_e = np.round(early_only/early_only.sum()*100,1).tolist()
  n_l = np.round(late/late.sum()*100,1).tolist()
  total_col = [total.sum()]
  non_exposed_col = [non_exposed.sum()]
  early_only_col = [early_only.sum()]
  late_col = [late.sum()]
  p_col = ['']
  for i in range(len(counts)):
    total_col.append((str(total[i])+' ('+str(n_t[i])+')'))
    non_exposed_col.append((str(non_exposed[i])+' ('+str(n_p[i])+')'))
    early_only_col.append((str(early_only[i])+' ('+str(n_e[i])+')'))
    late_col.append((str(late[i])+' ('+str(n_l[i])+')'))
    if i != len(counts):
      p_col.append('')
    else:
      p_col.append()
  newcount['Variable'] = variable_col
  newcount['Total'] = total_col
  newcount['Non_exposed'] = non_exposed_col
  newcount['Early_only'] = early_only_col
  newcount['Late'] = late_col
  result_df = pd.DataFrame(newcount)
  return result_df

In [0]:
categorical_variable(anytime_before, 'N06AX_exposure0')

In [0]:
from scipy.stats import chi2_contingency
from statsmodels.sandbox.stats.multicomp import multipletests
from itertools import combinations


# This module contains functions that wrap the standard chi2_contingency test from scipy.stats.,
# as well as additional corrections for multiple comparisons and post-hoc tests.
# The usage is as simple as chisq_and_posthoc_corrected(df) 

# Uploaded by Moran Neuhof, 2018


def get_asterisks_for_pval(p_val, alpha=0.05):
    """Receives the p-value and returns asterisks string."""
    if p_val > alpha:  # bigger than alpha
        p_text = "ns"
    # following the standards in biological publications
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text  # string of asterisks


def run_chisq_on_combination(df, combinations_tuple):
    """Receives a dataframe and a combinations tuple and returns p-value after performing chisq test."""
    assert len(combinations_tuple) == 2, "Combinations tuple is too long! Should be of size 2."
    new_df = df[(df.index == combinations_tuple[0]) | (df.index == combinations_tuple[1])]
    chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
    return p


def chisq_and_posthoc_corrected(df, column, correction_method='fdr_bh', alpha=0.05):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction).
    alpha: optional threshold for rejection (default: 0.05)
    correction_method: method used for mutiple comparisons correction. (default: 'fdr_bh').
    See statsmodels.sandbox.stats.multicomp.multipletests for elaboration."""
    crossdf = pd.crosstab(anytime_before['SSRI_exposure_group'], anytime_before[column])
    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(crossdf, correction=True)
    print("Chi2 result of the contingency table: {}, p-value: {}\n".format(chi2, p))
    
    # post-hoc test
    all_combinations = list(combinations(crossdf.index, 2))  # gathering all combinations for post-hoc chi2
    print("Post-hoc chi2 tests results:")
    p_vals = [run_chisq_on_combination(crossdf, comb) for comb in all_combinations]  # a list of all p-values
    # the list is in the same order of all_combinations
    outcome = {}
    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method=correction_method, alpha=alpha)[:2]
    outcome['entire'] = p
    outcome['(late, nonexposed)'] = corrected_p_vals[2]
    outcome['(early-only, nonexposed)'] = corrected_p_vals[1]
    outcome['(early-only, late)'] = corrected_p_vals[0]
    crossdf2 = pd.crosstab(df[column], df['preterm_category'])
    chi2, p2, dof, ex = chi2_contingency(crossdf2, correction=True)
    outcome['preterm_category'] = p2
    result_df = pd.DataFrame.from_dict(outcome, 'index').T
    return result_df 

In [0]:
new_df = crossdf[(crossdf.index == 'late') | (crossdf.index == 'non_exposed')]
chi2, p2, dof, ex = chi2_contingency(new_df , correction=True)
p2

In [0]:
run_chisq_on_combination(crossdf, ('late','non_exposed'))

In [0]:
crossdf = pd.crosstab(anytime_before['SSRI_exposure_group'], anytime_before['N06AB_exposure0'])
chi2, p2, dof, ex = chi2_contingency(crossdf, correction=True)
crossdf

In [0]:
chisq_and_posthoc_corrected(anytime_before, 'phq-4 category')

In [0]:
run_multiple_ttest(anytime_before, 'phq4_score')

In [0]:
print (*anytime_before.columns + '\n')

In [0]:
def run_ANOVA_test(y, df):
  import statsmodels.api as sm
  from statsmodels.formula.api import ols
  model = ols('{0} ~ C(SSRI_exposure_group)'.format(y), data=df).fit()
  anova_table = sm.stats.anova_lm(model, typ=2)
  return anova_table.iloc[0,3]
def run_ttest(df, column, comb, grouping):
  a = df.loc[df[grouping]==comb[0]][column]
  b = df.loc[df[grouping]==comb[1]][column]
  return stats.ttest_ind(a, b, equal_var=False, nan_policy = 'omit').pvalue
def run_multiple_ttest(df, column):
  all_combinations = list(combinations(set(df.SSRI_exposure_group), 2))
  p_vals = [run_ttest(df, column, comb, 'SSRI_exposure_group') for comb in all_combinations]
  reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh', alpha=0.05)[:2]
  outcome = {}
  outcome['ANOVA'] = run_ANOVA_test(column, df = df)
  outcome['(non_exposed,late)'] = corrected_p_vals[2]
  outcome['(non_exposed,early_only)'] = corrected_p_vals[0]
  outcome['(early_only,late)'] = corrected_p_vals[1]
  outcome['(PTB, non-PTB)'] = run_ttest(df, column, list(set(df.preterm_category)), 'preterm_category')
  result_df = pd.DataFrame.from_dict(outcome, 'index').T
  return result_df 
  

In [0]:
run_ttest(anytime_before, 'Parity', list(set(anytime_before.preterm_category)), 'preterm_category')

In [0]:
run_multiple_ttest(anytime_before, 'Preterm_history')

In [0]:
def run_ttest(df, column, comb):
  a = df.loc[df['SSRI_exposure_group']==comb[0]][column]
  b = df.loc[df['SSRI_exposure_group']==comb[1]][column]
  return stats.ttest_ind(a, b, equal_var=False, nan_policy = 'omit').pvalue

In [0]:
all_combinations[0][0]

In [0]:
def run_ANOVA_test(y, df):
  import statsmodels.api as sm
  from statsmodels.formula.api import ols
  model = ols('{0} ~ C(SSRI_exposure_group)'.format(y), data=df).fit()
  anova_table = sm.stats.anova_lm(model, typ=2)
  return anova_table.iloc[0,3]
def continuous_variable(cohort_pd, variable):
  newcount = {}
  counts = cohort_pd.groupby('SSRI_exposure_group').agg({variable:['count','mean','std']}).transpose()
  variable_col = ['total N', variable]
  source_col = []
  non_exposed = counts.iloc[:,2].to_numpy()
  late = counts.iloc[:,1].to_numpy()
  early_only = counts.iloc[:,0].to_numpy()
  total_col = [cohort_pd[variable].notnull().sum(), str(cohort_pd[variable].mean().round(1))+'±'+str(cohort_pd[variable].std().round(1))]
  non_exposed_col = [non_exposed[0],(str(non_exposed[1].round(1))+'±'+str(non_exposed[2].round(1)))]
  early_only_col = [early_only[0],(str(early_only[1].round(1))+'±'+str(early_only[2].round(1)))]
  late_col = [late[0],(str(late[1].round(1))+'±'+str(late[2].round(1)))]
  #p_col = ['', run_ANOVA_test(variable, cohort_pd)]
  newcount['Variable'] = variable_col
  newcount['Total'] = total_col
  newcount['Non_exposed'] = non_exposed_col
  newcount['Early_only'] = early_only_col
  newcount['Late'] = late_col
  #newcount['p-value'] = p_col
  result_df = pd.DataFrame(newcount)
  return result_df

# Population Statistics 

1. ANOVA test across exposure groups 
2. chi-square test across exposure groups

In [0]:
def run_ANOVA_test(y, x, df):
  import statsmodels.api as sm
  from statsmodels.formula.api import ols
  model = ols('{0} ~ C({1})'.format(y,x), data=df).fit()
  anova_table = sm.stats.anova_lm(model, typ=2)
  return anova_table

In [0]:
run_ANOVA_test('during_phq4_score', 'SSRI_exposure_group', anytime_before)

In [0]:
run_ANOVA_test('prior_phq9_score', 'SSRI_exposure_group', anytime_before)

In [0]:
run_ANOVA_test('prior_phq9_score', anytime_before)

In [0]:
continuous_variable(anytime_before, 'prior_phq9_score')